In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping,TensorBoard, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization, LeakyReLU 
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

import os
import numpy as np

# plotting
import matplotlib.pyplot as plt
import pathlib
from PIL import Image
import IPython.display as display

# to determine the most voted
import collections 

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [2]:
def get_label(file_path):
    # convert the path to a list of path components
    parts = tf.strings.split(file_path, os.path.sep)
    # The second to last is the class-directory
    return parts[-2] == classNames

def decode_img(img):
    # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_png(img, channels=3)
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    img = tf.image.convert_image_dtype(img, tf.float32)
    # resize the image to the desired size.
    return tf.image.resize(img, [32,32])

def get_bytes_and_label(file_path):
    label = get_label(file_path)
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

def show_batch(image_batch, label_batch):
    columns = 6
    rows = BATCH_SIZE / columns + 1  
    plt.figure(figsize=(10, 2 * rows))
    for n in range(BATCH_SIZE):
        ax = plt.subplot(int(rows), columns, n+1)
        plt.imshow((image_batch[n]))
        plt.title(classNames[label_batch[n]==1][0])
        plt.axis('off')
 

In [3]:
def prepare_callbacks(file_path):

    checkpointer = ModelCheckpoint(filepath= file_path, 
                               monitor = 'val_accuracy',
                               verbose=1, 
                               save_weights_only=True,
                               save_best_only=True)


    earlyStopper = EarlyStopping(monitor='val_loss', min_delta = 0.0001, patience = 15, verbose = 1)

    reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.000000001, verbose = 1)

    return [checkpointer, earlyStopper, reduceLR]

In [4]:
BATCH_SIZE = 128
IMAGE_SIZE = 32
NUM_MODELS = 9

In [5]:
data_dir = pathlib.Path('Final_Training/Images/')
  
classNames = np.array(os.listdir(data_dir))
NUM_CLASSES = len(classNames)
classNames

array(['00000', '00001', '00002', '00003', '00004', '00005', '00006',
       '00007', '00008', '00009', '00010', '00011', '00012', '00013',
       '00014', '00015', '00016', '00017', '00018', '00019', '00020',
       '00021', '00022', '00023', '00024', '00025', '00026', '00027',
       '00028', '00029', '00030', '00031', '00032', '00033', '00034',
       '00035', '00036', '00037', '00038', '00039', '00040', '00041',
       '00042'], dtype='<U5')

In [6]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

listset = tf.data.Dataset.list_files("Final_Training/Images/*/*.png")
dataset = listset.map(get_bytes_and_label, num_parallel_calls = AUTOTUNE)

t = next(iter(dataset))
print(t[0].shape, t[1].shape)

dataset_length = tf.data.experimental.cardinality(dataset).numpy()
print("Total images in dataset: ", dataset_length) 

(32, 32, 3) (43,)
Total images in dataset:  27930


In [7]:
val_listset = tf.data.Dataset.list_files("Final_Validation/Images/*/*.png")
val_dataset_length = val_listset.cardinality().numpy()
print("Total images in validatation dataset: ", val_dataset_length)

valset = val_listset.map(get_bytes_and_label, num_parallel_calls = AUTOTUNE)
valset = valset.cache()
valset = valset.shuffle(buffer_size = val_dataset_length)
valset = valset.batch(batch_size = BATCH_SIZE)
valset = valset.prefetch(buffer_size = AUTOTUNE)


test_listset = tf.data.Dataset.list_files("Final_Test/Images/*/*.png")
test_dataset_length = test_listset.cardinality().numpy()
print("Total images in test dataset: ", test_dataset_length)

testset = test_listset.map(get_bytes_and_label, num_parallel_calls = AUTOTUNE)
testset = testset.batch(batch_size = BATCH_SIZE)

Total images in validatation dataset:  11280
Total images in test dataset:  12630


In [8]:
def create_model(classCount, imgSize, channels):

    modelLogits = Sequential()
    
    modelLogits.add(Conv2D(128, (5, 5),
                input_shape=(imgSize, imgSize, channels)))         
    modelLogits.add(LeakyReLU(alpha=0.01))  
    modelLogits.add(BatchNormalization())
    modelLogits.add(Dropout(0.5)) 

    modelLogits.add(Conv2D(196, (5, 5) )) 
    modelLogits.add(LeakyReLU(alpha=0.01))
    modelLogits.add(MaxPooling2D(pool_size=(2, 2)))
    modelLogits.add(BatchNormalization())
    modelLogits.add(Dropout(0.5)) 

    modelLogits.add(Conv2D(256, (5, 5) ) )   
    modelLogits.add(LeakyReLU(alpha=0.01))
    modelLogits.add(MaxPooling2D(pool_size=(2, 2)))
    modelLogits.add(BatchNormalization())
    modelLogits.add(Dropout(0.5)) 
    
    modelLogits.add(Flatten())
    modelLogits.add(Dense(384))
    modelLogits.add(LeakyReLU(alpha=0.0))             
    modelLogits.add(Dropout(0.5)) 
    
    modelLogits.add(Dense(classCount))
    
    output = Activation('softmax')(modelLogits.output)

    model = tf.keras.Model(modelLogits.inputs, output)
    
    opt = Adam(learning_rate=0.0001)
    model.compile(optimizer = opt, loss='categorical_crossentropy', metrics=[ 'accuracy'])
    return model, modelLogits


### Data augmentation functions

In [9]:
import tensorflow_addons as tfa

def process_brightness(image, label):
    
    img = tf.clip_by_value(tfa.image.random_hsv_in_yiq(image, 0.0, 1.0, 1.0, 0.1, 3.0),0,1)
    return img, label

def process_saturation(image, label):
    
    img = tf.clip_by_value(tfa.image.random_hsv_in_yiq(image, 0.0, 1.0, 3.0, 1.0, 1.0),0,1)
    return img, label

def process_contrast(image, label):
    
    img = tf.clip_by_value(tf.image.random_contrast(image, lower=0.1, upper=3.0, seed=None), 0, 1)
    return img, label

def process_hue(image, label):
    
    img = tf.image.random_hue(image, max_delta=0.2, seed=None)
    return img, label

def process_rotate(image, label):
    
    img = tfa.image.rotate(image, tf.random.uniform(shape=(), minval=-0.175, maxval=0.175))
    return img, label

def process_shear(image, label):
    
    img = tfa.image.rotate(image, tf.random.uniform(shape=(), minval=-0.175, maxval=0.175))
    sx = tf.random.uniform(shape=(), minval=-0.1, maxval=0.1, dtype=tf.dtypes.float32)
    img = tfa.image.transform(img, [1, sx, -sx*32,   0,1,0,  0,0])
    return img, label

def process_translate(image, label):

    img = tfa.image.rotate(image, tf.random.uniform(shape=(), minval=-0.175, maxval=0.175))
    tx = tf.random.uniform(shape=(), minval=-3, maxval=3, dtype=tf.dtypes.float32)
    ty = tf.random.uniform(shape=(), minval=-3, maxval=3, dtype=tf.dtypes.float32)  
    img = tfa.image.translate(img, [tx,ty])
    return img, label

def process_crop(image, label):
    
    c = tf.random.uniform(shape=(), minval=24, maxval=32, dtype=tf.dtypes.float32)
    img = tf.image.random_crop(image, size=[c,c,3])
    img = tf.image.resize(img ,size= [32,32])
    return img, label

### Ensemble functions

In [10]:
def train_models(train, val,file_path_prefix, new_length):
    models = []
    histories = []
    
    for i in range(NUM_MODELS):

        model, modelL = create_model(NUM_CLASSES,IMAGE_SIZE,3)

        callbacks = prepare_callbacks(f'{file_path_prefix}_{i:02}/cp.ckpt')
        
        hist = model.fit(train, steps_per_epoch = new_length / BATCH_SIZE,
                              epochs=100, 
                              validation_data = val, 
                              callbacks = callbacks)

        models.append([model, modelL])
        histories.append(hist)
    
    return models,histories


def create_models():

    models = []
    
    for i in range(NUM_MODELS):
        model, modelL = create_model(NUM_CLASSES,IMAGE_SIZE,3)
        models.append([model, modelL])

    return models


def load_weights(models, file_path_prefix):
    for i in range(NUM_MODELS):
        file_path = f'{file_path_prefix}_{i:02}/cp.ckpt'
        models[i][0].load_weights(file_path)
        models[i][0].save('ensemble/temp.hdf5')
        models[i][1].load_weights('ensemble/temp.hdf5', by_name = True)


def evaluate_models(models):
    
    accuracy = 0
    
    for i in range(NUM_MODELS):
        eval = models[i][0].evaluate(testset, verbose = 2)
        accuracy += eval[1]
        
    print(f'average accuracy: {(accuracy/NUM_MODELS)*100:.3f}')

    
def get_labels_logits_and_preds(models):

    preds = [[] for _ in range(NUM_MODELS) ]
    logits = [[] for _ in range(NUM_MODELS)]
    labels = []
    for images, labs in testset.take(-1):

        labels.extend(labs.numpy())
        for i in range(NUM_MODELS):

            preds[i].extend(models[i][0].predict(images))
            logits[i].extend(models[i][1].predict(images))

    labels = [np.argmax(i) for i in labels]  
    
    return labels, logits, preds


def get_class_preds(preds):

    class_preds = []

    for i in range(test_dataset_length):

        c = []
        for m in range(NUM_MODELS):

            c.append(np.argmax(preds[m][i]))
        class_preds.append(c)
        
    return class_preds


def get_class_from_sum_of_logits(logits):

    sum_logits = []

    for i in range(test_dataset_length):

        log = logits[0][i]
        for m in range(1, NUM_MODELS):
            log = np.add(log, logits[m][i])
        sum_logits.append(np.argmax(log))
    return(sum_logits)


def get_stats(labels, class_preds, class_logits):

    all_correct = 0
    all_incorrect = 0
    maj_vote = 0
    maj_wrong = 0
    tie = 0
    count = 0
    log_ok = 0
    log_ko = 0

    for k in range(test_dataset_length):

        counter = collections.Counter(class_preds[k])
        if len(counter) == 1:
            if counter.most_common(1)[0][0] == labels[k]:
                all_correct += 1
            else:
                all_incorrect += 1
        else:
            aux = counter.most_common(2)
            if aux[0][1] > aux[1][1] and aux[0][0] == labels[k]:
                maj_vote += 1
            if aux[0][1] > aux[1][1] and aux[0][0] != labels[k]:
                maj_wrong += 1
            elif aux[0][1] == aux[1][1]:
                tie += 1
        if class_logits[k] == labels[k]:
            log_ok += 1
        else:
            log_ko += 1
        count += 1 
        
    return [count, all_correct, all_incorrect, maj_vote, tie, maj_wrong, log_ok, log_ko]

### Dataset augmentation

In [11]:
dataV1 = dataset
# color ops
dataV1 = dataV1.map(process_brightness)
dataV1 = dataV1.concatenate(dataset.map(process_contrast))
dataV1 = dataV1.concatenate(dataset.map(process_hue))
dataV1 = dataV1.concatenate(dataset.map(process_saturation))

#geometry ops
dataV1 = dataV1.concatenate(dataset.map(process_rotate))
dataV1 = dataV1.concatenate(dataset.map(process_shear))
dataV1 = dataV1.concatenate(dataset.map(process_translate))
dataV1 = dataV1.concatenate(dataset.map(process_crop))

# number of dataset augmentation functions used
dataset_updated_length = dataset_length * 8

dataV1 = dataV1.cache()
dataV1 = dataV1.shuffle(buffer_size = (dataset_updated_length))
dataV1 = dataV1.batch(batch_size = BATCH_SIZE)
dataV1 = dataV1.prefetch(buffer_size = AUTOTUNE)
dataV1 = dataV1.repeat()

In [12]:
file_path_prefix = 'ensemble/model_V1'
models_V1, histories_V1 = train_models(dataV1, valset, file_path_prefix, dataset_updated_length)

Epoch 1/100
1745/1745 [==============================] - 305s 109ms/step - loss: 1.3625 - accuracy: 0.6500 - val_loss: 0.1558 - val_accuracy: 0.9576

Epoch 00001: val_accuracy improved from -inf to 0.95762, saving model to ensemble/model_V1_00\cp.ckpt
Epoch 2/100
1745/1745 [==============================] - 185s 106ms/step - loss: 0.3663 - accuracy: 0.8962 - val_loss: 0.0724 - val_accuracy: 0.9771

Epoch 00002: val_accuracy improved from 0.95762 to 0.97713, saving model to ensemble/model_V1_00\cp.ckpt
Epoch 3/100
1745/1745 [==============================] - 184s 105ms/step - loss: 0.1984 - accuracy: 0.9431 - val_loss: 0.0456 - val_accuracy: 0.9859

Epoch 00003: val_accuracy improved from 0.97713 to 0.98590, saving model to ensemble/model_V1_00\cp.ckpt
Epoch 4/100
1745/1745 [==============================] - 185s 106ms/step - loss: 0.1286 - accuracy: 0.9621 - val_loss: 0.0357 - val_accuracy: 0.9887

Epoch 00004: val_accuracy improved from 0.98590 to 0.98865, saving model to ensemble/mod

KeyboardInterrupt: 

In [14]:
load_weights(models_V1, file_path_prefix)
evaluate_models(models_V1)
labels_V1, logits_V1, preds_V1 = get_labels_logits_and_preds(models_V1)
class_preds_V1 = get_class_preds(preds_V1)
class_logits_V1 = get_class_from_sum_of_logits(logits_V1)    

res = get_stats(labels_V1, class_preds_V1, class_logits_V1)
print(res, res[6]/res[0])

99/99 - 24s - loss: 0.0394 - accuracy: 0.9877
99/99 - 4s - loss: 0.0417 - accuracy: 0.9872
99/99 - 4s - loss: 0.0392 - accuracy: 0.9892
average accuracy: 98.804
[12630, 12386, 47, 102, 17, 78, 12513, 117] 0.9907363420427553


In [15]:
res = get_stats(labels_V1, class_preds_V1, class_logits_V1)
print('[count, all_correct, all_incorrect, maj_vote, tie, maj_wrong, log_ok, log_ko], accuracy')
print(res, res[6]/res[0])

[count, all_correct, all_incorrect, maj_vote, tie, maj_wrong, log_ok, log_ko], accuracy
[12630, 12386, 47, 102, 17, 78, 12513, 117] 0.9907363420427553
